**SLP vs MLP**

In this activity you will compare the performance of single-layer perceptrons (SLPs) versus multilayer perceptrons (MLPs) on two different classification tasks: MNIST digit recognition and playing tic-tac-toe.

To get started, run the setup script below. This script will load several different datasets into memory so it may take awhile to finish...be patient!

In [16]:
import os
#from google.colab import drive
import pickle
import gdown

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output
import random

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)  # For Apple Silicon

# ------- uncomment to load full MNIST dataset through torchvision and downsamlpe to 1/5 original size
# # Load MNIST dataset
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# full_train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
# # Subsample 1/5 of the data while keeping class balance
# indices_by_class = {i: [] for i in range(10)}
# for idx, (_, label) in enumerate(full_train_dataset):
#     indices_by_class[label] = indices_by_class.get(label, []) + [idx]
# sampled_indices = []
# for i in range(10):
#     sampled_indices.extend(np.random.choice(indices_by_class[i], len(indices_by_class[i]) // 5, replace=False))
# train_dataset = torch.utils.data.Subset(full_train_dataset, sampled_indices)
# test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# ------- uncomment to save a reduced MNIST dataset to a file
# # Save train_dataset and test_dataset to files
# output_folder = "/content/drive/My Drive/Public/datasets"
# train_dataset_file = os.path.join(output_folder, 'MNIST_train_dataset_small.pkl')
# test_dataset_file = os.path.join(output_folder, 'MNIST_test_dataset_small.pkl')
# with open(train_dataset_file, 'wb') as f:
#     pickle.dump(train_dataset, f)
# with open(test_dataset_file, 'wb') as f:
#     pickle.dump(test_dataset, f)


# ---------- uncomment to load reduced MNIST data from blair Public folder ----------
# Define Google Drive file IDs (Extract these from shareable links)
train_file_id = "1bnBkKnOxhQJlMcPItHCKWnO1wTenRjEd"
test_file_id = "1a-YMPj6rPkEfaP5F9k-5b4Gp-bQtQXLT"
output_folder = "./content/datasets"
os.makedirs(output_folder, exist_ok=True)  # Ensure directory exists
train_dataset_file = os.path.join(output_folder, 'MNIST_train_dataset_small.pkl')
test_dataset_file = os.path.join(output_folder, 'MNIST_test_dataset_small.pkl')
gdown.download(f"https://drive.google.com/uc?id={train_file_id}", train_dataset_file, quiet=False)
gdown.download(f"https://drive.google.com/uc?id={test_file_id}", test_dataset_file, quiet=False)
with open(train_dataset_file, 'rb') as f:
    train_dataset = pickle.load(f)
with open(test_dataset_file, 'rb') as f:
    test_dataset = pickle.load(f)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# ---------- uncomment to dowload tic tac toe data from blair Public folder ------------
file_id = "1hRgI_d0H8yDatEqWQxQTRXeasYYXtAIS"  # Replace with actual file ID
output_folder = "./content/datasets"
os.makedirs(output_folder, exist_ok=True)  # Ensure directory exists
data_file = os.path.join(output_folder, "tic_tac_toe_dataset.npz")
gdown.download(f"https://drive.google.com/uc?id={file_id}", data_file, quiet=False)
data = np.load(data_file)
X = data['X']
y = data['y']
X_flat = X.reshape(X.shape[0], -1)
y_labels = np.argmax(y, axis=1)
Xttt_train, yttt_train = X_flat, y_labels
X_train_tensor = torch.tensor(Xttt_train, dtype=torch.float32)
y_train_tensor = torch.tensor(yttt_train, dtype=torch.long)

#----- create the tic tac toe game ---------
# Define Tic-Tac-Toe Board
board = np.zeros(9)  # 1 for X, -1 for O, 0 for empty
user_symbol = 1  # X
ai_symbol = -1  # O

# Load or define model
input_size = 9
num_classes = 9

# Generate buttons for user input
buttons = [widgets.Button(description=' ', layout=widgets.Layout(width='50px', height='50px')) for _ in range(9)]

# Display the Tic-Tac-Toe board
def display_board():
    clear_output(wait=True)
    board_layout = widgets.GridBox(children=buttons, layout=widgets.Layout(grid_template_columns='repeat(3, 50px)'))
    display(board_layout)

# Handle user move
def on_button_click(i):
    def click_callback(b):
        global game_over
        if board[i] == 0 and not game_over:  # If the spot is empty and the game is not over
            board[i] = user_symbol
            buttons[i].description = 'X'
            buttons[i].disabled = True
            if check_game_status():
                return  # Stop AI move if game is over
            ai_move()
            check_game_status()
    return click_callback

# AI move
def ai_move():
    global game_over
    if game_over:
        return  # Prevent AI from moving if game is over

    board_tensor = torch.tensor(board, dtype=torch.float32).unsqueeze(0)
    logits = model(board_tensor).detach().numpy().flatten()
    empty_indices = [i for i in range(9) if board[i] == 0]
    if empty_indices:
        best_move = max(empty_indices, key=lambda idx: logits[idx])
        board[best_move] = ai_symbol
        buttons[best_move].description = 'O'
        buttons[best_move].disabled = True

# Check for win/loss/draw
def check_game_status():
    global game_over
    winning_combinations = [(0,1,2), (3,4,5), (6,7,8), (0,3,6), (1,4,7), (2,5,8), (0,4,8), (2,4,6)]
    for combo in winning_combinations:
        line = [board[i] for i in combo]
        if sum(line) == 3:
            print("You win!")
            disable_buttons()
            game_over = True
            return True
        elif sum(line) == -3:
            print("AI wins!")
            disable_buttons()
            game_over = True
            return True
    if 0 not in board:
        print("It's a draw!")
        disable_buttons()
        game_over = True
        return True
    return False

# Disable buttons after game ends
def disable_buttons():
    for btn in buttons:
        btn.disabled = True

# Assign button callbacks
for i in range(9):
    buttons[i].on_click(on_button_click(i))




Downloading...
From: https://drive.google.com/uc?id=1bnBkKnOxhQJlMcPItHCKWnO1wTenRjEd
To: /Users/rianbutala/rians-projects/Coding/Other/cosmos/mlp/content/datasets/MNIST_train_dataset_small.pkl
100%|██████████| 47.7M/47.7M [00:09<00:00, 5.24MB/s]
Downloading...
From: https://drive.google.com/uc?id=1a-YMPj6rPkEfaP5F9k-5b4Gp-bQtQXLT
To: /Users/rianbutala/rians-projects/Coding/Other/cosmos/mlp/content/datasets/MNIST_test_dataset_small.pkl
100%|██████████| 7.92M/7.92M [00:00<00:00, 13.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hRgI_d0H8yDatEqWQxQTRXeasYYXtAIS
To: /Users/rianbutala/rians-projects/Coding/Other/cosmos/mlp/content/datasets/tic_tac_toe_dataset.npz
100%|██████████| 26.6k/26.6k [00:00<00:00, 3.88MB/s]


**QUESTION 1 PART A:**

Let's start by training an SLP on the MNIST dataset to recognize hand written digits:


>![](https://drive.google.com/uc?id=1OGQ6tS81rmoYQ7sA1E5bq6iE4aErB0aS)


When you run the script below, the SLP will be trained to classify inputs as one of the digits 0-9. The network will be trained for 10 epochs (that is, 10 complete passes through the entire training set).You will then see a report of performance results.

In [17]:

# Prompt user for hidden layer size
hidden_units = 0 #int(input("Enter the number of units in the MLP hidden layer (0 for SLP): "))

# Define MLP or SLP model dynamically
class NeuralNet(nn.Module):
    def __init__(self, hidden_units):
        super(NeuralNet, self).__init__()
        if hidden_units > 0:
            self.fc1 = nn.Linear(28*28, hidden_units)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_units, 10)
        else:
            self.fc1 = nn.Linear(28*28, 10)  # Single-layer perceptron (SLP)
            self.relu = None

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        if self.relu:
            x = self.relu(x)
            x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = NeuralNet(hidden_units).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    correct, total, running_loss = 0, 0, 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {correct/total*100:.2f}%")

# Evaluate the model
model.eval()
correct, total = 0, 0
class_correct = [0] * 10
class_total = [0] * 10
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        for i in range(len(labels)):
            label = labels[i].item()
            class_correct[label] += (predicted[i] == labels[i]).item()
            class_total[label] += 1
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print accuracy for each digit
print("\nValidation Accuracy for each digit:")
for i in range(10):
    acc = 100 * class_correct[i] / class_total[i]
    print(f"Digit {i}: {acc:.2f}%")

# Print overall accuracy
print(f"Overall Test Accuracy: {correct / total * 100:.2f}%")

slpota = correct / total * 100


Epoch 1/10, Loss: 0.8099, Train Accuracy: 77.85%
Epoch 2/10, Loss: 0.4344, Train Accuracy: 87.45%
Epoch 3/10, Loss: 0.3780, Train Accuracy: 89.04%
Epoch 4/10, Loss: 0.3429, Train Accuracy: 90.25%
Epoch 5/10, Loss: 0.3282, Train Accuracy: 90.29%
Epoch 6/10, Loss: 0.3164, Train Accuracy: 90.82%
Epoch 7/10, Loss: 0.3039, Train Accuracy: 91.00%
Epoch 8/10, Loss: 0.2977, Train Accuracy: 91.13%
Epoch 9/10, Loss: 0.2887, Train Accuracy: 91.56%
Epoch 10/10, Loss: 0.2866, Train Accuracy: 91.66%

Validation Accuracy for each digit:
Digit 0: 95.41%
Digit 1: 97.53%
Digit 2: 87.11%
Digit 3: 88.02%
Digit 4: 92.36%
Digit 5: 87.78%
Digit 6: 89.98%
Digit 7: 92.80%
Digit 8: 88.09%
Digit 9: 87.12%
Overall Test Accuracy: 90.73%


**QUESTION 1 PART B:**

Now let's train an MLP with one hidden layer on the same MNIST dataset:

>![](https://drive.google.com/uc?id=1XHSay4fcjHbpoFYn8jR_9QO2nmRN3DBY)

When you run the script below, you will be prompted to enter the number of units for the MLP's hidden layer. Then the network will be trained for 10 epochs (that is, 10 complete passes through the entire training set) using the number of hidden units you selected.

You should find that by using more hidden units, you can improve the Overall Test Accuracy. Keep running the script until you find a number of hidden units that increases the Overall Test Accuracy by at least 5% higher than the result you obtained above for the SLP with no hidden layer (for example, if your SLP achieved 88% accuracy then train the MLP to at least 93% accuracy).  

In [27]:
# Prompt user for hidden layer size
hidden_units = int(input("Enter the number of units in the MLP hidden layer (0 for SLP): "))

# Define MLP or SLP model dynamically
class NeuralNet(nn.Module):
    def __init__(self, hidden_units):
        super(NeuralNet, self).__init__()
        if hidden_units > 0:
            self.fc1 = nn.Linear(28*28, hidden_units)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_units, 10)
        else:
            self.fc1 = nn.Linear(28*28, 10)  # Single-layer perceptron (SLP)
            self.relu = None

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        if self.relu:
            x = self.relu(x)
            x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = NeuralNet(hidden_units).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    correct, total, running_loss = 0, 0, 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {correct/total*100:.2f}%")

# Evaluate the model
model.eval()
correct, total = 0, 0
class_correct = [0] * 10
class_total = [0] * 10
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        for i in range(len(labels)):
            label = labels[i].item()
            class_correct[label] += (predicted[i] == labels[i]).item()
            class_total[label] += 1
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print accuracy for each digit
print("\nValidation Accuracy for each digit:")
for i in range(10):
    acc = 100 * class_correct[i] / class_total[i]
    print(f"Digit {i}: {acc:.2f}%")

# Print overall accuracy
print(f"Overall Test Accuracy: {correct / total * 100:.2f}%")




Epoch 1/10, Loss: 0.7188, Train Accuracy: 79.78%
Epoch 2/10, Loss: 0.3738, Train Accuracy: 88.77%
Epoch 3/10, Loss: 0.3101, Train Accuracy: 91.01%
Epoch 4/10, Loss: 0.2803, Train Accuracy: 91.46%
Epoch 5/10, Loss: 0.2481, Train Accuracy: 92.64%
Epoch 6/10, Loss: 0.2236, Train Accuracy: 93.30%
Epoch 7/10, Loss: 0.2007, Train Accuracy: 94.01%
Epoch 8/10, Loss: 0.1754, Train Accuracy: 94.86%
Epoch 9/10, Loss: 0.1572, Train Accuracy: 95.47%
Epoch 10/10, Loss: 0.1398, Train Accuracy: 95.90%

Validation Accuracy for each digit:
Digit 0: 99.08%
Digit 1: 97.71%
Digit 2: 92.93%
Digit 3: 91.09%
Digit 4: 94.70%
Digit 5: 90.58%
Digit 6: 94.78%
Digit 7: 94.07%
Digit 8: 94.97%
Digit 9: 91.18%
Overall Test Accuracy: 94.17%


**QUESTION 2 PART A**

Now we will train an SLP to play tic-tac-toe by converting board positions into images:
>![](https://drive.google.com/uc?id=11Db63-PeVkNDJuveRFExzZhS0npbFEBL)

There are only 6,617 unique tic-tac-toe board positions, so we can train the network on ALL possible board positions. This means that we will only have a training set and no testing set.

When you run the script below, the network will be trained for 1000 epochs You will then be able to play games of tic-tac-toe against the trained SLP network.

In [19]:
# Prompt user for hidden layer size
hidden_units = 0 #int(input("Enter the number of units in the hidden layer (0 for SLP): "))

# Define Model
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_units, num_classes):
        super(NeuralNet, self).__init__()
        if hidden_units > 0:
            self.fc1 = nn.Linear(input_size, hidden_units)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_units, num_classes)
        else:
            self.fc1 = nn.Linear(input_size, num_classes)  # SLP case
            self.relu = None

    def forward(self, x):
        x = self.fc1(x)
        if self.relu:
            x = self.relu(x)
            x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
input_size = X_train_tensor.shape[1]
num_classes = len(torch.unique(y_train_tensor))
model = NeuralNet(input_size, hidden_units, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_train_tensor).sum().item() / y_train_tensor.size(0) * 100
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")

Q2_slpacc = accuracy


Epoch [100/1000], Loss: 2.0365, Accuracy: 26.36%
Epoch [200/1000], Loss: 2.0331, Accuracy: 26.25%
Epoch [300/1000], Loss: 2.0329, Accuracy: 26.24%
Epoch [400/1000], Loss: 2.0329, Accuracy: 26.22%
Epoch [500/1000], Loss: 2.0329, Accuracy: 26.24%
Epoch [600/1000], Loss: 2.0329, Accuracy: 26.25%
Epoch [700/1000], Loss: 2.0329, Accuracy: 26.25%
Epoch [800/1000], Loss: 2.0329, Accuracy: 26.25%
Epoch [900/1000], Loss: 2.0329, Accuracy: 26.25%
Epoch [1000/1000], Loss: 2.0329, Accuracy: 26.25%


**QUESTION 2 PART B**

What level of accuracy did the SLP network achieve? How well do you think this network will be able to play tic-tac-toe? Let's find out! Run the script below to play games against the SLP bot. Click in a square to make your move. Play as many times as you want!

In [20]:
game_over = False  # Flag to track if the game is over
board = np.zeros(9)  # 1 for X, -1 for O, 0 for empty
for btn in buttons:
    btn.description = ' '
    btn.disabled = False
display_board()

GridBox(children=(Button(description=' ', layout=Layout(height='50px', width='50px'), style=ButtonStyle()), Bu…

**QUESTION 2 PART C**

Now let's train an MLP with one hidden layer to play tic-tac-toe:

>![](https://drive.google.com/uc?id=1aMu1Geb6RwMnUlvnBFUWmB2m8Oat3dVU)

When you run the script you will be prompted to enter the number of hidden layer units. Can you train the network to be 100% accurate with enough hidden units?

After training to near 100% accuracy, try playing against the AI again below.

In [21]:
# Prompt user for hidden layer size
hidden_units = int(input("Enter the number of units in the hidden layer (0 for SLP): "))
input_size = X_train_tensor.shape[1]
num_classes = len(torch.unique(y_train_tensor))
model = NeuralNet(input_size, hidden_units, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Training loop
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_train_tensor).sum().item() / y_train_tensor.size(0) * 100
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")



Epoch [100/1000], Loss: 0.2004, Accuracy: 94.98%
Epoch [200/1000], Loss: 0.0934, Accuracy: 98.59%
Epoch [300/1000], Loss: 0.0470, Accuracy: 99.64%
Epoch [400/1000], Loss: 0.0251, Accuracy: 99.97%
Epoch [500/1000], Loss: 0.0147, Accuracy: 100.00%
Epoch [600/1000], Loss: 0.0094, Accuracy: 100.00%
Epoch [700/1000], Loss: 0.0064, Accuracy: 100.00%
Epoch [800/1000], Loss: 0.0047, Accuracy: 100.00%
Epoch [900/1000], Loss: 0.0035, Accuracy: 100.00%
Epoch [1000/1000], Loss: 0.0027, Accuracy: 100.00%


Play against the AI again by running this script:

In [26]:
game_over = False  # Flag to track if the game is over
board = np.zeros(9)  # 1 for X, -1 for O, 0 for empty
for btn in buttons:
    btn.description = ' '
    btn.disabled = False
display_board()

GridBox(children=(Button(description=' ', layout=Layout(height='50px', width='50px'), style=ButtonStyle()), Bu…